In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import csv
import numpy as np
import tensorflow as tf
from PIL import Image
import pandas as pd
import random
import keras
from tcn import TCN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

from datetime import datetime
import socket


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

os.environ['CUDA_HOME'] = '/usr/lib/cuda'
cuda_lib_path = os.path.join(os.environ['CUDA_HOME'], 'lib64')
if 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH'] = f"{cuda_lib_path}:{os.environ['LD_LIBRARY_PATH']}"
else:
    os.environ['LD_LIBRARY_PATH'] = cuda_lib_path

print("LD_LIBRARY_PATH set to:", os.environ['LD_LIBRARY_PATH'])
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("CUDA_HOME:", os.environ.get('CUDA_HOME'))
print("LD_LIBRARY_PATH:", os.environ.get('LD_LIBRARY_PATH'))
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))
print("GPUs:", tf.config.list_physical_devices('GPU'))

LD_LIBRARY_PATH set to: /usr/lib/cuda/lib64
TensorFlow version: 2.17.0
CUDA_HOME: /usr/lib/cuda
LD_LIBRARY_PATH: /usr/lib/cuda/lib64
Num GPUs Available: 1
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1720929313.955252   60859 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720929314.023086   60859 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720929314.023413   60859 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
add_flipped = False
WINDOW_SIZE = 10
BATCH_SIZE = 32
patience = 100
Partition_Of_DataSets = 1

GAUSSIAN_NOISE_SD = 0.1
MAX_ROTATION_ANGLE_NOISE = 0
STORM_INTENSITY_NOISE = 0

In [4]:
PIXEL_TO_METER_SCALE = 13.913
image = Image.open("data/map/hkust_4f.jpg")
image = image.resize((int(image.size[0] / PIXEL_TO_METER_SCALE), int(image.size[1] / PIXEL_TO_METER_SCALE))).transpose(Image.FLIP_TOP_BOTTOM)

In [5]:
def process_csv_file(csv_path, image_size, pixel_to_meter_scale):
    path_data = {'x':[], 'y':[], "Bv":[], "Bh":[], "Bp":[]}
    
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Skip the header
        
        for row in reader:
            x = float(row[3]) / pixel_to_meter_scale
            y = - float(row[4]) / pixel_to_meter_scale + image_size[1]
            Bv = float(row[0])
            Bh = float(row[1])
            Bp = float(row[2])
            
            path_data["x"].append(x)
            path_data["y"].append(y)
            path_data["Bv"].append(Bv)
            path_data["Bh"].append(Bh)
            path_data["Bp"].append(Bp)
    
    return path_data


train_raw_data = []
data_path = os.path.join(".", "data", "formatted", "HKUST_4F", "training data")

for root, _, files in os.walk(data_path):
    for file in files:
        if file.endswith('.csv'):
            train_raw_data.append(process_csv_file(os.path.join(root, file), image.size, PIXEL_TO_METER_SCALE))

test_raw_data = []
test_data_path = os.path.join(".", "data", "formatted", "HKUST_4F", "testing data")

for root, _, files in os.walk(test_data_path):
    for file in files:
        if file.endswith('.csv'):
            test_raw_data.append(process_csv_file(os.path.join(root, file), image.size, PIXEL_TO_METER_SCALE))

In [6]:
def print_data_info(data, name):
    total_length = sum(len(d['x']) for d in data)
    individual_lengths = [len(d['x']) for d in data]
    num_trajectories = len(data)
    
    print(f"{name} data:")
    print(f"  Total length: {total_length}")
    print(f"  Individual lengths: {individual_lengths}")
    print(f"  Number of trajectories: {num_trajectories}")
    print()


print_data_info(train_raw_data, "Train Raw")
print_data_info(test_raw_data, "Test Raw")


Train Raw data:
  Total length: 170013
  Individual lengths: [678, 625, 646, 669, 665, 666, 643, 686, 645, 677, 685, 690, 677, 660, 4373, 4318, 4239, 4339, 4291, 4350, 4414, 4294, 4330, 4296, 4314, 4422, 4302, 4489, 5611, 5713, 5732, 6119, 5714, 5604, 5454, 6268, 5822, 5592, 5412, 5563, 5503, 6328, 680, 685, 682, 649, 685, 710, 680, 673, 665, 684, 664, 672, 686, 670, 692, 703, 713, 675, 687, 740, 763, 700, 682, 705, 736, 710, 745, 759]
  Number of trajectories: 70

Test Raw data:
  Total length: 24589
  Individual lengths: [750, 5958, 658, 713, 662, 4313, 6572, 4282, 681]
  Number of trajectories: 9



In [7]:
train_data = []
train_labels = []
valid_data = []
valid_labels = []
test_data = []
test_labels = []



def split_data(data, partition=4):
    result = []
    for traj in data:
        for i in range(partition):
            new_traj = {key: value[i::partition] for key, value in traj.items()}
            result.append(new_traj)
    return result
def prepare_datasets(train_data, test_data, partition=4, val_split=0.2):
    train_processed = split_data(train_data, partition)
    test_processed = split_data(test_data, partition)
    train_data, val_data = train_test_split(train_processed, test_size=val_split, random_state=42)
    return train_data, val_data, test_processed

train_data, valid_data, test_data = prepare_datasets(train_raw_data, test_raw_data, partition=Partition_Of_DataSets)

In [8]:
print_data_info(train_data, "Train")
print_data_info(valid_data, "Validation")
print_data_info(test_data, "Test")

Train data:
  Total length: 142903
  Individual lengths: [763, 5732, 713, 6268, 703, 710, 4239, 5454, 680, 5611, 686, 672, 5503, 682, 685, 4350, 692, 5563, 4422, 5412, 660, 665, 669, 4339, 645, 670, 643, 5822, 682, 759, 745, 4318, 4489, 6328, 4302, 680, 4314, 675, 700, 690, 5714, 736, 740, 5592, 5713, 685, 705, 625, 664, 4294, 646, 4296, 4414, 687, 4373, 684]
  Number of trajectories: 56

Validation data:
  Total length: 27110
  Individual lengths: [4330, 678, 673, 665, 686, 4291, 685, 5604, 649, 677, 6119, 677, 710, 666]
  Number of trajectories: 14

Test data:
  Total length: 24589
  Individual lengths: [750, 5958, 658, 713, 662, 4313, 6572, 4282, 681]
  Number of trajectories: 9



In [9]:
def rotate_vector(v, theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.dot(np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]]), v)

def magnetic_storm(t, amplitude, frequency):
    return amplitude * np.sin(2 * np.pi * frequency * t) * np.exp(-t)

def process_dataset(dataset, is_training=False, add_flipped=False, gaussian_noise_sd=0, max_rotation_angle=0, storm_intensity=0):
    data, labels = [], []
    
    for d in dataset:
        
        if len(d['Bv']) < WINDOW_SIZE:
            continue
        
        for window_idx in range(len(d['Bv']) - WINDOW_SIZE + 1):
            window = slice(window_idx, window_idx + WINDOW_SIZE)
            B_data = np.column_stack((d['Bv'][window], d['Bh'][window], d['Bp'][window]))

            if is_training:
                if gaussian_noise_sd > 0:
                    B_data += np.random.normal(0, gaussian_noise_sd, B_data.shape)
                
                if max_rotation_angle > 0:
                    theta = np.random.uniform(-max_rotation_angle, max_rotation_angle)
                    B_data = np.apply_along_axis(lambda v: rotate_vector(v, theta), 1, B_data)

                if storm_intensity > 0:
                    t = np.linspace(0, 1, WINDOW_SIZE)
                    storm = magnetic_storm(t, storm_intensity, np.random.uniform(0.5, 2.0))
                    B_data += storm[:, np.newaxis]

            data.append(B_data)
            labels.append([d['x'][window.stop-1], d['y'][window.stop-1]])

            if add_flipped:
                data.append(np.flip(B_data, axis=0))
                labels.append([d['x'][window.start], d['y'][window.start]])
    
    return np.array(data), np.array(labels)


In [10]:
# Process datasets
train_data, train_labels = process_dataset(train_data, is_training=True)
valid_data, valid_labels = process_dataset(valid_data, is_training=False)
test_data, test_labels = process_dataset(
    test_data, 
    is_training=False, 
    add_flipped=False,
    gaussian_noise_sd=GAUSSIAN_NOISE_SD,
    max_rotation_angle=MAX_ROTATION_ANGLE_NOISE,
    storm_intensity=STORM_INTENSITY_NOISE,
)

print(f"shape of train_data: {train_data.shape}")
print(f"shape of train_labels: {train_labels.shape}")
print(f"shape of valid_data: {valid_data.shape}")
print(f"shape of valid_labels: {valid_labels.shape}")
print(f"shape of test_data: {test_data.shape}")
print(f"shape of test_labels: {test_labels.shape}")

# Convert data to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_data, valid_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))


train_dataset = train_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

shape of train_data: (142399, 10, 3)
shape of train_labels: (142399, 2)
shape of valid_data: (26984, 10, 3)
shape of valid_labels: (26984, 2)
shape of test_data: (24508, 10, 3)
shape of test_labels: (24508, 2)


I0000 00:00:1720929317.555497   60859 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720929317.555790   60859 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720929317.555977   60859 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720929317.749330   60859 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [11]:
NB_FILTERS = 8
DILATIONS = [1, 2, 4, 8, 16]
DENSE_UNITS = [16, 8, 4, 2]
LEAKY_RELU_ALPHA = 0.1
LOSS = 'mae'

In [12]:
def create_model(nb_filters, dilations, dense_units, leaky_relu_alpha, loss):
    # Default Adam optimizer
    optimizer = tf.keras.optimizers.Adam()

    model = keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=(WINDOW_SIZE, 3)),
        TCN(input_shape=(WINDOW_SIZE, 3), nb_filters=nb_filters, return_sequences=True, dilations=dilations),
        keras.layers.Flatten(),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(leaky_relu_alpha)
    ])
    for units in dense_units[:-1]:  # All but the last dense layer
        model.add(keras.layers.Dense(units))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.LeakyReLU(leaky_relu_alpha))
    model.add(keras.layers.Dense(dense_units[-1]))
    
    # Compile the model with additional metrics
    model.compile(optimizer=optimizer, 
                  loss=loss, 
                  metrics=[
                      keras.metrics.MeanAbsoluteError(),
                      keras.metrics.MeanSquaredError(),
                      keras.metrics.RootMeanSquaredError(),
                      keras.metrics.MeanAbsolutePercentageError()
                  ])
    
    return model

# Create the model using the defined hyperparameters
model = create_model(
    nb_filters=NB_FILTERS,
    dilations=DILATIONS,
    dense_units=DENSE_UNITS,
    leaky_relu_alpha=LEAKY_RELU_ALPHA,
    loss=LOSS
)

# model.summary()

/root/myenv/lib/python3.12/site-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/root/myenv/lib/python3.12/site-packages/tcn/tcn.py:227: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


In [13]:
from tensorflow.keras.callbacks import TensorBoard
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
hostname = socket.gethostname()
log_dir = f'logs/experiment_{current_time}_{hostname}'
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Subset training parameters
subset_train_size = 1000
subset_val_size = 200
subset_batch_size = 4
subset_epochs = 100
subset_model_path = 'saved_model/subset_model.keras'

# Create subsets
train_indices = np.random.choice(train_data.shape[0], size=subset_train_size, replace=False)
subset_train_data = train_data[train_indices]
subset_train_labels = train_labels[train_indices]

val_indices = np.random.choice(valid_data.shape[0], size=subset_val_size, replace=False)
subset_val_data = valid_data[val_indices]
subset_val_labels = valid_labels[val_indices]

# Train on subset
subset_checkpoint = ModelCheckpoint(subset_model_path, monitor='val_mean_absolute_error', save_best_only=True)

subset_history = model.fit(
    subset_train_data, subset_train_labels,
    validation_data=(subset_val_data, subset_val_labels),
    callbacks=[tensorboard_callback, subset_checkpoint],
    batch_size=subset_batch_size,
    epochs=subset_epochs,
)

Epoch 1/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 96.9651 - mean_absolute_error: 96.9651 - mean_absolute_percentage_error: 57.5961 - mean_squared_error: 12219.1621 - root_mean_squared_error: 110.4912 - val_loss: 70.4214 - val_mean_absolute_error: 70.4214 - val_mean_absolute_percentage_error: 37.4476 - val_mean_squared_error: 7624.7998 - val_root_mean_squared_error: 87.3201
Epoch 2/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 83.5815 - mean_absolute_error: 83.5815 - mean_absolute_percentage_error: 49.3642 - mean_squared_error: 9867.4707 - root_mean_squared_error: 99.3137 - val_loss: 52.3876 - val_mean_absolute_error: 52.3876 - val_mean_absolute_percentage_error: 25.7379 - val_mean_squared_error: 5430.0996 - val_root_mean_squared_error: 73.6892
Epoch 3/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 72.8809 - mean_absolute_error: 72.8809 - mean_absolute_percentage_error: 41.3273 - mean_squared_error: 8598.4785 - root_mean_squared_error: 92.6771 - val_loss: 46

In [ ]:
# Main training parameters
main_batch_size = 32
main_epochs = 100
main_model_path = 'saved_model/main_model.keras'


# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_mean_absolute_error',
    patience=patience,
    restore_best_weights=True
)

# Train on main dataset
main_checkpoint = ModelCheckpoint(main_model_path, monitor='val_mean_absolute_error', save_best_only=True)

main_history = model.fit(
    train_data, train_labels,
    validation_data=(valid_data, valid_labels),
    callbacks=[tensorboard_callback, main_checkpoint,early_stopping],
    batch_size=main_batch_size,
    initial_learning_rate=main_initial_learning_rate,
    epochs=main_epochs
)

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf

# Load the saved model
loaded_model = load_model('saved_model/main_model.keras')

# Overall evaluation
loss, mae, mse, rmse, mape = loaded_model.evaluate(test_data, test_labels)
print(f"Overall Loss: {loss:.4f}")
print(f"Overall MAE: {mae:.4f}")
print(f"Overall MSE: {mse:.4f}")
print(f"Overall RMSE: {rmse:.4f}")
print(f"Overall MAPE: {mape:.4f}")

# Individual sample evaluation
predictions = loaded_model.predict(test_data)
errors = np.abs(predictions - test_labels).flatten()

# Error statistics
err68, err95 = np.percentile(errors, [68, 95])
print(f"68th percentile error (err68): {err68:.4f}")
print(f"95th percentile error (err95): {err95:.4f}")

# Log metrics to TensorBoard
with tf.summary.create_file_writer(log_dir).as_default():
    tf.summary.scalar('Evaluation/Loss', loss, step=0)
    tf.summary.scalar('Evaluation/MAE', mae, step=0)
    tf.summary.scalar('Evaluation/MSE', mse, step=0)
    tf.summary.scalar('Evaluation/RMSE', rmse, step=0)
    tf.summary.scalar('Evaluation/MAPE', mape, step=0)
    
    for i, error in enumerate(errors):
        tf.summary.scalar('Individual_MAE', error, step=i)
    
    tf.summary.scalar('Errors/err68', err68, step=0)
    tf.summary.scalar('Errors/err95', err95, step=0)
    tf.summary.histogram('Error_Distribution', errors, step=0)

print(f"TensorBoard logs saved to: {log_dir}")


In [ ]:

# Additional evaluation metrics
from sklearn.metrics import r2_score, mean_absolute_percentage_error

r2 = r2_score(test_labels, predictions)
mape = mean_absolute_percentage_error(test_labels, predictions)

print(f"R-squared: {r2:.4f}")
print(f"MAPE: {mape:.4f}")

# Visualize predictions vs actual values
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(test_labels, predictions, alpha=0.5)
plt.plot([test_labels.min(), test_labels.max()], [test_labels.min(), test_labels.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predictions vs Actual Values')
plt.tight_layout()
plt.show()

# Plot error distribution
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50)
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.title('Error Distribution')
plt.tight_layout()
plt.show()